# Let me hear the music

# Content <a id='back'></a>

* [Introduction](#intro)
* [Stage 1. Data Description](#data_review)
    * [Conclusions](#data_review_conclusions)
* [Stage 2. Data Preprocessing](#data_preprocessing)
    * [2.1 Header Style](#header_style)
    * [2.2 Missing Values](#missing_values)
    * [2.3 Duplicates](#duplicates)
    * [2.4 Conclusions](#data_preprocessing_conclusions)
* [Stage 3. Hypothesis Testing](#hypothesis)
    * [3.1 Hypothesis 1: Male and Female User Activity in the Two Cities](#activity)
* [Conclusions](#end)

 ## Introduction <a id='intro'></a>

The purpose of this work is to analyze data to extract valuable information and facilitate informed decision-making. The process includes several stages, such as data overview, preprocessing, and hypothesis testing.

In any research, it is essential to formulate hypotheses that can then be evaluated. Some of these hypotheses are accepted, while others are rejected. To make sound decisions, a company must be able to identify whether its assumptions are correct.

In this project, the musical preferences of users in the cities of Springfield and Shelbyville were compared. Real-world online music streaming data was used to test the hypothesis and analyze user behavior in both locations.

### Objective:
To evaluate the following hypothesis:
1. User activity differs by day of the week and city.

### Stages
User behavior data is stored in the file `/datasets/music_project_en.csv`.

First, data quality will be assessed to determine the extent of potential problems. Critical aspects will then be addressed during preprocessing.

Project Stages:
1. Data Description.
2. Data Preprocessing.
3. Hypothesis Testing.

[Volver a Contenidos](#back)

## Step 1. Data Description <a id='data_review'></a>

Open and review the data.

In [35]:
import pandas as pd

In [36]:
# Read the music_project_en.csv file and store it in df
df = pd.read_csv('./datasets/music_project_en.csv')

Muestra las 10 primeras filas de la tabla:

In [37]:
# Get the first 10 rows from table df
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

In [38]:
# Get general information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Table Notes: It contains seven columns, each storing the same data type: `object`.

According to the documentation:
- ` userID`: User ID;
- `Track`: Song title;
- `artist`: Artist name;
- `genre`: Track genre;
- `City`: User city;
- `time`: Exact time the song was played;
- `Day`: Day of the week.

Style issues with table headers:
1. Some headers are uppercase, others lowercase.
2. There are spaces in some headers.
3. Snake_case 'userID' as it would be better read as 'user_id'.

### Other observations

1. There is categorical data. The data is descriptive; there are no numbers in the table. The data is available for statistics and hypothesis testing.

2. There is sufficient data, as there is a column to determine the day the music was played and another column to determine the city.

3. There is missing data in three columns, and there appears to be incorrect data, as one username has a shorter string than the others, and the gender includes 'rusrap' and 'ruspop'. This data looks strange.

### Header Style <a id='header_style'></a>
Display table headers (column names)

In [39]:
# Displays column names
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Change table headings according to good style rules:
* All characters must be lowercase.
* Remove spaces.
* If the name has multiple words, use snake_case.

In [40]:
# Loop through headers making everything lowercase
new_col_names = []
for low in df.columns:
    name_lowered = low.lower()
    new_col_names.append(name_lowered)
    
df.columns = new_col_names    
print(df.columns)

Index(['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day'], dtype='object')


In [41]:
# Loop through headers removing spaces
new_col_names_2 = []
for old_names in new_col_names:
    name_stripped = old_names.strip()
    new_col_names_2.append(name_stripped)

df.columns = new_col_names_2
print(df.columns)

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


In [42]:
# Rename the "userid" column
df.rename(columns = {'userid':'user_id'}, inplace=True)
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


In [43]:
# Check the result: list of headers
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


[Volver a Contenidos](#back)

### Missing values <a id='missing_values'></a>

In [44]:
# Calculate the number of missing values
print(df.isna().sum())

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


Not all missing values affect the research. For example, missing values in `track` and `artist` are not crucial. They can simply be replaced with default values like the string `unknown`.

But missing values in `genre` can affect the comparison between the musical preferences of Springfield and Shelbyville. In real life, it would be useful to know the reasons for missing data and attempt to recover them. But that opportunity doesn't exist in this project. So the choice was to:
* fill these missing values with a default value.
* assess how much the missing values might impact the results.

In [45]:
# Loop through headers replacing missing values with 'unknown'
cols = ['track', 'artist', 'genre']
    
for col in cols:
    df[col] = df[col].fillna('unknown')


In [46]:
# Count missing values
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


[Volver a Contenidos](#back)

### Duplicates <a id='duplicates'></a>

In [47]:
#Count duplicates
print(df.duplicated().sum())

3826


In [48]:
# Remove duplicates
df = df.drop_duplicates()

In [49]:
# Check for duplicates
print(df.duplicated().sum())

0


To identify and correct implicit duplicates in the `genre` column, you must first view the unique names of the genres. To do this:

* Extract the genre column from the DataFrame.

* Then, apply the method that returns the unique values present in that column.

* Finally, sort the columns alphabetically to facilitate review and detect inconsistencies.

In [50]:
# Inspeccionar los nombres de géneros únicos
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'triphop', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk',
       'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'salsa', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'tropical', 'contemporary', 'new', 'soul', 'holiday',
       'german', 'jpop', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'korean', 'numetal', 'vocal', 'estrada', 'tango',
       'loungeelectronic', 'classicmetal', 'dubstep', 'club', 'deep',
       'southern', 'black', 'folkrock', 

To correct implicit duplicates of the hiphop genre, such as hip, hop, and hip-hop, a function called replace_wrong_genres() must be defined, which takes two parameters:

* wrong_genres: A list of incorrect or alternate names for the genre.

* correct_genre: A text string representing the correct name to use as a replacement.

In the body of the function, the list of incorrect genres is traversed using a for loop. In each iteration, the 'genre' column of the DataFrame df is accessed and the replace method is applied to replace the incorrect value with the correct name. This standardizes the genre names in the table.

In [ ]:
# Function to replace implicit duplicates
def replace_wrong_genres(df, wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genres, correct_genre)
        return df

In [ ]:
# Remove implicit duplicates
print(replace_wrong_genres(df, ['hip', 'hop', 'hip-hop'], 'hiphop'))

        user_id                              track            artist  \
0      FFB692EC                  Kamigata To Boots  The Mass Missile   
1      55204538        Delayed Because of Accident  Andreas Rönnberg   
2        20EC38                  Funiculì funiculà       Mario Lanza   
3      A3DD03C9              Dragons in the Sunset        Fire + Ice   
4      E2DC1FAE                        Soul People        Space Echo   
...         ...                                ...               ...   
65074  729CBB09                            My Name            McLean   
65075  D08D4A55  Maybe One Day (feat. Black Spade)       Blu & Exile   
65076  C5E3A0D5                          Jalopiina           unknown   
65077  321D0506                      Freight Train     Chas McDevitt   
65078  3A64EF84          Tell Me Sweet Little Lies      Monica Lopez   

            genre         city      time        day  
0            rock  Shelbyville  20:28:33  Wednesday  
1            rock  Springfi

In [ ]:
# Checking for implicit duplicates
print(df['genre'].unique())

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hiphop' 'jazz' 'postrock'
 'latin' 'classical' 'metal' 'reggae' 'triphop' 'blues' 'instrumental'
 'rusrock' 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'salsa' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'tropical' 'contemporary' 'new' 'soul' 'holiday'
 'german' 'jpop' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'korean'
 'numetal' 'vocal' 'estrada' 'tango' 'loungeelectronic' 'classicmetal'
 'dubstep' 'club' 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french'
 'disco' 'religious' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'swing' 'breaks' 'eurofolk'
 'stone

[Volver a Contenidos](#back)

## Step 3. Hypothesis Testing <a id='hypothesis'></a>

### Hypothesis: User activity differs by day of the week and city. <a id='activity'></a>

The hypothesis states that there are differences in the way users in Springfield and Shelbyville consume music. To test this, data from three days of the week were used: Monday, Wednesday, and Friday.

* Group users by city.
* Compare the number of songs each group played on Monday, Wednesday, and Friday.

In [ ]:
# Count the songs played in each city
print(df.groupby(by='city')['track'].count())

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64


In Springfield they listen to more songs than in Shelbyville.

Group the data by day of the week and find the number of songs played on Monday, Wednesday, and Friday.

In [ ]:
# Calculate the songs played on each of the three days
print(df.groupby(by='day')['track'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


More songs are played on Fridays.

Write a function that can count entries based on both criteria simultaneously.

Create the `number_tracks()` function to calculate the number of songs played on a given day **and** city. The function must accept two parameters:

- `day`: A day of the week to filter on. For example, `'Monday'`.
- `city`: A city to filter on. For example, `'Springfield'`.

After filtering the data by two criteria, count the number of values in the 'user_id' column in the resulting table. This count represents the number of entries being searched.

In [56]:
# Declara la función number_tracks() con dos parámetros: day= y city=.
def number_track(df, day, city):
    tracks_by_day = df[df['day'] == day] # Almacena las filas del DataFrame donde el valor en la columna 'day' es igual al parámetro day=

    tracks_by_city = tracks_by_day[tracks_by_day['city'] == city] # Filtra las filas donde el valor en la columna 'city' es igual al parámetro city=

    user_count = tracks_by_city['user_id'].count() # Extrae la columna 'user_id' de la tabla filtrada y aplica el método count()

    return user_count # Devolve el número de valores de la columna 'user_id'

Llama a `number_tracks()` seis veces, cambiando los valores de los parámetros para que recuperes los datos de ambas ciudades para cada uno de los tres días.

In [57]:
# El número de canciones reproducidas en Springfield el lunes
print(number_track(df, 'Monday', 'Springfield'))

15740


In [58]:
# El número de canciones reproducidas en Shelbyville el lunes
print(number_track(df, 'Monday', 'Shelbyville'))

5614


In [59]:
# El número de canciones reproducidas en Springfield el miércoles
print(number_track(df, 'Wednesday', 'Springfield'))

11056


In [60]:
# El número de canciones reproducidas en Shelbyville el miércoles
print(number_track(df, 'Wednesday', 'Shelbyville'))

7003


In [61]:
# El número de canciones reproducidas en Springfield el viernes
print(number_track(df, 'Friday', 'Springfield'))

15945


In [62]:
# El número de canciones reproducidas en Shelbyville el viernes
print(number_track(df, 'Friday', 'Shelbyville'))

5895


**Conclusiones**

`Comenta si la hipótesis es correcta o se debe rechazar. Explica tu razonamiento.` Considero que es medianamente aceptable, al parecer no depende tanto del día de la semana, pero si hay una diferencia muy marcada dependiendo de la ciudad.

[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

`Resume aquí tus conclusiones sobre la hipótesis.` Parece que la hipótesis no es del todo correcta, ya que en Springfield no hay una variación tan grande en el número de canciones entre el lunes y el viernes, mientras que en Shelbyville el día que reproducen más canciones es el miércoles, por lo tanto, parece que no depende del día de la semana. En cambio, se encontró una diferencia muy grande entre la cantidad de canciones en Shelbyville y en Springfield. El 69.7% de canciones son de Springfield, por lo tanto se puede concluir que la actividad de los usuarios si depende de la ciudad.

[Volver a Contenidos](#back)